# Part 1

In [1]:
filename = "inputs/12-19.txt"
with open(filename, "r") as f:
    data = f.read()

In [2]:
import re
from functools import lru_cache

In [3]:
blueprints = {}
for l in data.splitlines():
    bp, or_or, cl_or, ob_or, ob_cl, ge_or, ge_ob = (int(v) for v in re.findall("\d+", l))
    blueprints[bp] = {
        "ore": {
            "ore": or_or,
        },
        "clay": {
            "ore": cl_or,
        },
        "obsidian": {
            "ore": ob_or,
            "clay": ob_cl,
        },
        "geode": {
            "ore": ge_or,
            "obsidian": ge_ob,
        },
    }
def bp_to_tuple(bp):
    return tuple(v for d in bp.values() for v in d.values())

In [4]:
@lru_cache(maxsize=None)
def geodes(t, state, bp):
    (ore_r, clay_r, obs_r, geo_r, ore, clay, obs) = state
    (ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs) = bp
    # if we can't possibly build another geode before t = 1 (excluded)
    if obs + (obs_r + (clay + (clay_r + (ore + ore_r * (t-4))//clay_ore) * (t-3))//obs_clay) * (t-2) < geo_obs:
        return t * geo_r
    else:
        next_ore = ore + ore_r
        next_clay = clay + clay_r
        next_obs = obs + obs_r
        # delaying geode robots is never profitable
        if ore >= geo_ore and obs >= geo_obs:
            return geo_r + geodes(
                t - 1,
                (ore_r, clay_r, obs_r, geo_r + 1, next_ore - geo_ore, next_clay, next_obs - geo_obs),
                bp
            )
        # possibly do nothing
        geos = [geodes(
            t - 1,
            (ore_r, clay_r, obs_r, geo_r, next_ore, next_clay, next_obs),
            bp
        )]
        # build an ore robot (up to a maximum)
        if ore >= ore_ore and ore_r < max([clay_ore, obs_ore, geo_ore]):
             geos.append(geodes(
                t - 1,
                (ore_r + 1, clay_r, obs_r, geo_r, next_ore - ore_ore, next_clay, next_obs),
                bp
            ))
        # build a clay robot (up to a maximum)
        if ore >= clay_ore and clay_r < obs_clay:
            geos.append(geodes(
                t - 1,
                (ore_r, clay_r + 1, obs_r, geo_r, next_ore - clay_ore, next_clay, next_obs),
                bp
            ))
        # build an obsidian robot (up to a maximum)
        if ore >= obs_ore and clay >= obs_clay and obs_r < geo_obs:
            geos.append(geodes(
                t - 1,
                (ore_r, clay_r, obs_r + 1, geo_r, next_ore - obs_ore, next_clay - obs_clay, next_obs),
                bp
            ))
        return geo_r + max(geos)

In [5]:
%%time
s = 0
for i, bp in blueprints.items():
    geo = geodes(24, (1, 0, 0, 0, 0, 0, 0), bp_to_tuple(bp))
    s += i * geo
s

CPU times: user 9.82 s, sys: 204 ms, total: 10 s
Wall time: 10 s


1092

# Part 2

In [6]:
%%time
p = 1
for i, bp in list(blueprints.items())[:3]:
    geo = geodes(32, (1, 0, 0, 0, 0, 0, 0), bp_to_tuple(bp))
    p *= geo
p

CPU times: user 2min 6s, sys: 1.44 s, total: 2min 7s
Wall time: 2min 7s


3542